In [1]:
from __future__ import print_function
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import keras
import warnings
import tensorflow as tf
import seaborn as sns
import sklearn
import random
import math
import time


from lime.lime_tabular import RecurrentTabularExplainer
from tqdm import tqdm
from tqdm import tqdm
from sklearn.metrics import mean_squared_error, r2_score 
from sklearn.model_selection import GroupKFold
from sklearn import preprocessing
from keras import backend as K
from sklearn.preprocessing import MinMaxScaler , StandardScaler
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM, Activation, GRU
from scipy import optimize
from tensorflow.keras import optimizers
from tensorflow.keras.callbacks import EarlyStopping


from sp_modif.model_function import *
from sp_modif.methods import *
from sp_modif.data_prep import *
from sp_modif.evaluator import *
from sp_modif.SHAP import *
from sp_modif.L2X import *
from methods import *

%matplotlib inline
warnings.filterwarnings('ignore')

SEED = 0
def set_seed(seed=SEED):
    os.environ['PYTHONHASHSEED'] = str(SEED)
    random.seed(SEED)
    np.random.seed(SEED)
    tf.random.set_seed(SEED)

# Appeler la fonction pour fixer le seed
set_seed(SEED)

In [2]:
# Load data
# Load data and preprocess
train, test, y_test = prepare_data('FD004.txt')
print(train.shape, test.shape, y_test.shape)
sensor_names = ['T20','T24','T30','T50','P20','P15','P30','Nf','Nc','epr','Ps30','phi',
                    'NRf','NRc','BPR','farB','htBleed','Nf_dmd','PCNfR_dmd','W31','W32']

remaining_sensors = ['T24','T30','T50', 'P15', 'P30','Nf','Nc', 'epr','Ps30','phi',
                    'NRf','NRc','BPR', 'farB','htBleed','W31','W32']
drop_sensors = [element for element in sensor_names if element not in remaining_sensors]

rul_piecewise = 120
train['RUL'].clip(upper=rul_piecewise, inplace=True)

(61249, 27) (41214, 26) (248, 1)


In [3]:
def model_lstm_1layer(input_shape, nodes_per_layer, dropout, activation):
    
    cb = keras.callbacks.EarlyStopping(monitor='loss', patience=4)
    model = Sequential()
    model.add(LSTM(units = nodes_per_layer, activation=activation, 
                  input_shape=input_shape))
    model.add(Dropout(dropout))
    model.add(Dense(256))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error',
                  optimizer=Adam(learning_rate=0.001))
    # model.save_weights(weights_file)

    return model

In [4]:
%%time
results_all = pd.DataFrame()
for SEED in range(5):  
    tf.random.set_seed(SEED)
    mse = []
    R2_val = []
    RMSE = []
    score_val = []
    
    # 0.20	[64]	0.3	tanh	32	25
    
    # parameter's sample
    # weights_file = "weights_file_lstm_optimalmodel_all.h5"
    alpha = 0.3
    sequence_length = 40
    epochs = 20
    nodes_per_layer = [64]
    dropout = 0.2
    activation = 'tanh'
    batch_size = 32
    remaining_sensors = remaining_sensors
    # create model
    input_shape = (sequence_length, len(remaining_sensors))
    model = model_lstm_1layer(input_shape, nodes_per_layer[0], dropout, activation)
    
    # Data prepration
    X_train_interim, X_test_interim = prep_data(train, test, drop_sensors, remaining_sensors, alpha)

    # create sequences train, test
    train_array = gen_data_wrapper(X_train_interim, sequence_length,remaining_sensors)
    label_array = gen_label_wrapper(X_train_interim, sequence_length, ['RUL'])

    test_gen = (list(gen_test_data(X_test_interim[X_test_interim['Unit']==unit_nr], sequence_length,remaining_sensors, -99.))
               for unit_nr in X_test_interim['Unit'].unique())
    
    test_array = np.concatenate(list(test_gen)).astype(np.float32)
    test_rul = rul_piecewise_fct(y_test,rul_piecewise)
    print(train_array.shape, label_array.shape, test_array.shape)
            
    # Model fitting
    cb = EarlyStopping(monitor='val_loss', patience=4, restore_best_weights=True)
    with tf.device('/device:GPU:0'):
        start_time = time.time()
        model.compile(loss='mean_squared_error', optimizer=Adam(learning_rate=0.001))
        weights_file = model.get_weights()
        model.set_weights(weights_file)  # reset optimizer and node weights before every training iteration
        history = model.fit(train_array, label_array,
                                validation_data=(test_array, test_rul),
                                epochs=epochs,
                                batch_size=batch_size,
                                callbacks=[cb],
                                verbose=1)
        end_time = time.time()
        training_time = end_time - start_time
        mse.append(history.history['val_loss'][-1])

        y_hat_val_split = model.predict(test_array)
        R2_val.append(r2_score(test_rul, y_hat_val_split))
        RMSE.append(np.sqrt(mean_squared_error(test_rul, y_hat_val_split)))
        score_val.append(compute_s_score(test_rul, y_hat_val_split))
            
        
    #  append results
    d = {'RMSE' :np.mean(RMSE), 'std_RMSE' :np.std(RMSE),
         'S_score' :np.mean(score_val), 'std_S_score' :np.std(score_val),
         'MSE':np.mean(mse), 'std_MSE':np.std(mse),
         'nodes':str(nodes_per_layer), 'dropout':dropout, 
         'activation':activation, 'batch_size':batch_size, 'TW' : sequence_length,
         'time':training_time}

#     results = results.append(pd.DataFrame(d, index=[0]), ignore_index=True)
    results_all = pd.concat([results_all, pd.DataFrame(d, index=[0])], ignore_index=True)
    results_all.to_csv('results/all/fd004.csv')


(51538, 40, 17) (51538, 1) (248, 40, 17)
Epoch 1/20

1611/1611 [==============================] - 20s 11ms/step - loss: 769.6389 - val_loss: 471.7919
Epoch 2/20
1611/1611 [==============================] - 18s 11ms/step - loss: 297.4277 - val_loss: 250.2302
Epoch 3/20
1611/1611 [==============================] - 18s 11ms/step - loss: 249.4327 - val_loss: 221.5461
Epoch 4/20
1611/1611 [==============================] - 19s 12ms/step - loss: 231.5601 - val_loss: 217.8264
Epoch 5/20
1611/1611 [==============================] - 21s 13ms/step - loss: 215.5899 - val_loss: 192.3274
Epoch 6/20
1611/1611 [==============================] - 23s 14ms/step - loss: 200.9751 - val_loss: 201.6725
Epoch 7/20
1611/1611 [==============================] - 21s 13ms/step - loss: 189.2692 - val_loss: 184.5225
Epoch 8/20
1611/1611 [==============================] - 22s 13ms/step - loss: 183.2186 - val_loss: 180.1667
Epoch 9/20
1611/1611 [==============================] - 23s 14ms/step - loss: 176.4012 - val_

In [5]:
results_all

,RMSE,std_RMSE,S_score,std_S_score,MSE,std_MSE,nodes,dropout,activation,batch_size,TW,time
0,12.392929,0.0,687.639444,0.0,153.584702,0.0,[64],0.2,tanh,32,40,398.625060
1,12.359558,0.0,657.331306,0.0,153.000153,0.0,[64],0.2,tanh,32,40,387.112779
2,13.238767,0.0,875.264684,0.0,175.264954,0.0,[64],0.2,tanh,32,40,409.568153
3,13.174353,0.0,621.174271,0.0,173.563568,0.0,[64],0.2,tanh,32,40,340.484180
4,13.405710,0.0,775.865868,0.0,179.989532,0.0,[64],0.2,tanh,32,40,233.254431


### FD002 <a class="anchor" id="fd002"></a>

In [6]:
# Load data
# Load data and preprocess
train, test, y_test = prepare_data('FD002.txt')
print(train.shape, test.shape, y_test.shape)
sensor_names = ['T20','T24','T30','T50','P20','P15','P30','Nf','Nc','epr','Ps30','phi',
                    'NRf','NRc','BPR','farB','htBleed','Nf_dmd','PCNfR_dmd','W31','W32']

remaining_sensors = ['T24','T30','T50', 'P15', 'P30','Nf','Nc', 'epr','Ps30','phi',
                    'NRf','NRc','BPR','htBleed','W31','W32']
drop_sensors = [element for element in sensor_names if element not in remaining_sensors]

rul_piecewise = 125
train['RUL'].clip(upper=rul_piecewise, inplace=True)

(53759, 27) (33991, 26) (259, 1)


In [7]:
%%time
results_all002 = pd.DataFrame()
for SEED in range(5):  
    tf.random.set_seed(SEED)
    mse = []
    R2_val = []
    RMSE = []
    score_val = []
    
    # parameter's sample
    # weights_file = "weights_file_lstm_optimalmodel_all.h5"
    alpha = 0.2
    sequence_length = 40
    epochs = 20
    nodes_per_layer = [32]
    dropout = 0.1
    activation = 'tanh'
    batch_size = 128
    remaining_sensors = remaining_sensors
    # create model
    input_shape = (sequence_length, len(remaining_sensors))
    model = model_lstm_1layer(input_shape, nodes_per_layer[0], dropout, activation)
    
    # Data prepration
    X_train_interim, X_test_interim = prep_data(train, test, drop_sensors, remaining_sensors, alpha)

    # create sequences train, test
    train_array = gen_data_wrapper(X_train_interim, sequence_length,remaining_sensors)
    label_array = gen_label_wrapper(X_train_interim, sequence_length, ['RUL'])

    test_gen = (list(gen_test_data(X_test_interim[X_test_interim['Unit']==unit_nr], sequence_length,remaining_sensors, -99.))
               for unit_nr in X_test_interim['Unit'].unique())
    
    test_array = np.concatenate(list(test_gen)).astype(np.float32)
    test_rul = rul_piecewise_fct(y_test,rul_piecewise)
    print(train_array.shape, label_array.shape, test_array.shape)
            
    # Model fitting
    cb = EarlyStopping(monitor='val_loss', patience=4, restore_best_weights=True)
    with tf.device('/device:GPU:0'):
        start_time = time.time()
        model.compile(loss='mean_squared_error', optimizer=Adam(learning_rate=0.001))
        weights_file = model.get_weights()
        model.set_weights(weights_file)  # reset optimizer and node weights before every training iteration
        history = model.fit(train_array, label_array,
                                validation_data=(test_array, test_rul),
                                epochs=epochs,
                                batch_size=batch_size,
                                callbacks=[cb],
                                verbose=1)
        end_time = time.time()
        training_time = end_time - start_time
        mse.append(history.history['val_loss'][-1])

        y_hat_val_split = model.predict(test_array)
        R2_val.append(r2_score(test_rul, y_hat_val_split))
        RMSE.append(np.sqrt(mean_squared_error(test_rul, y_hat_val_split)))
        score_val.append(compute_s_score(test_rul, y_hat_val_split))
            
        
    #  append results
    d = {'RMSE' :np.mean(RMSE), 'std_RMSE' :np.std(RMSE),
         'S_score' :np.mean(score_val), 'std_S_score' :np.std(score_val),
         'MSE':np.mean(mse), 'std_MSE':np.std(mse),
         'nodes':str(nodes_per_layer), 'dropout':dropout, 
         'activation':activation, 'batch_size':batch_size, 'TW' : sequence_length,
         'time':training_time}

#     results = results.append(pd.DataFrame(d, index=[0]), ignore_index=True)
    results_all002 = pd.concat([results_all002, pd.DataFrame(d, index=[0])], ignore_index=True)
    results_all002.to_csv('results/all/fd002.csv')

(43619, 40, 16) (43619, 1) (259, 40, 16)
Epoch 1/20
341/341 [==============================] - 6s 13ms/step - loss: 2284.7844 - val_loss: 1184.4271
Epoch 2/20
341/341 [==============================] - 4s 11ms/step - loss: 489.2794 - val_loss: 232.4056
Epoch 3/20
341/341 [==============================] - 3s 10ms/step - loss: 277.6627 - val_loss: 208.3045
Epoch 4/20
341/341 [==============================] - 3s 10ms/step - loss: 252.8305 - val_loss: 209.8648
Epoch 5/20
341/341 [==============================] - 3s 10ms/step - loss: 239.3363 - val_loss: 197.7292
Epoch 6/20
341/341 [==============================] - 3s 9ms/step - loss: 235.1313 - val_loss: 200.9615
Epoch 7/20
341/341 [==============================] - 3s 10ms/step - loss: 233.2259 - val_loss: 186.3433
Epoch 8/20
341/341 [==============================] - 3s 10ms/step - loss: 230.1735 - val_loss: 191.8853
Epoch 9/20
341/341 [==============================] - 3s 10ms/step - loss: 224.1112 - val_loss: 196.1663
Epoch 10/20
3

In [8]:
results_all002

,RMSE,std_RMSE,S_score,std_S_score,MSE,std_MSE,nodes,dropout,activation,batch_size,TW,time
0,13.650763,0.0,838.278069,0.0,192.241714,0.0,[32],0.1,tanh,128,40,39.600502
1,13.073473,0.0,855.127888,0.0,171.446854,0.0,[32],0.1,tanh,128,40,64.197236
2,13.785293,0.0,782.038338,0.0,202.758621,0.0,[32],0.1,tanh,128,40,45.513574
3,13.298704,0.0,744.324630,0.0,183.460220,0.0,[32],0.1,tanh,128,40,41.246588
4,13.122668,0.0,893.482596,0.0,208.057388,0.0,[32],0.1,tanh,128,40,50.774455


### FD003 <a class="anchor" id="fd003"></a>

In [9]:
# Load data
# Load data and preprocess
train, test, y_test = prepare_data('FD002.txt')
print(train.shape, test.shape, y_test.shape)
sensor_names = ['T20','T24','T30','T50','P20','P15','P30','Nf','Nc','epr','Ps30','phi',
                    'NRf','NRc','BPR','farB','htBleed','Nf_dmd','PCNfR_dmd','W31','W32']

remaining_sensors = ['T24','T30','T50', 'P15', 'P30','Nf','Nc', 'epr','Ps30','phi',
                    'NRf','NRc','BPR','htBleed','W31','W32']
drop_sensors = [element for element in sensor_names if element not in remaining_sensors]

rul_piecewise = 125
train['RUL'].clip(upper=rul_piecewise, inplace=True)

(53759, 27) (33991, 26) (259, 1)


In [10]:
%%time
results_all003 = pd.DataFrame()
for SEED in range(5):  
    tf.random.set_seed(SEED)
    mse = []
    R2_val = []
    RMSE = []
    score_val = []
    
    # parameter's sample
    # weights_file = "weights_file_lstm_optimalmodel_all.h5"
    alpha = 0.1
    sequence_length = 35
    epochs = 20
    nodes_per_layer = [64]
    dropout = 0.2
    activation = 'tanh'
    batch_size = 32
    remaining_sensors = remaining_sensors
    # create model
    input_shape = (sequence_length, len(remaining_sensors))
    model = model_lstm_1layer(input_shape, nodes_per_layer[0], dropout,
                             activation)
    
    # Data prepration
    X_train_interim, X_test_interim = prep_data(train, test, drop_sensors, remaining_sensors, alpha)

    # create sequences train, test
    train_array = gen_data_wrapper(X_train_interim, sequence_length,remaining_sensors)
    label_array = gen_label_wrapper(X_train_interim, sequence_length, ['RUL'])

    test_gen = (list(gen_test_data(X_test_interim[X_test_interim['Unit']==unit_nr], sequence_length,remaining_sensors, -99.))
               for unit_nr in X_test_interim['Unit'].unique())
    
    test_array = np.concatenate(list(test_gen)).astype(np.float32)
    test_rul = rul_piecewise_fct(y_test,rul_piecewise)
    print(train_array.shape, label_array.shape, test_array.shape)
            
    # Model fitting
    cb = EarlyStopping(monitor='val_loss', patience=4, restore_best_weights=True)
    with tf.device('/device:GPU:0'):
        start_time = time.time()
        model.compile(loss='mean_squared_error', optimizer=Adam(learning_rate=0.001))
        weights_file = model.get_weights()
        model.set_weights(weights_file)  # reset optimizer and node weights before every training iteration
        history = model.fit(train_array, label_array,
                                validation_data=(test_array, test_rul),
                                epochs=epochs,
                                batch_size=batch_size,
                                callbacks=[cb],
                                verbose=1)
        end_time = time.time()
        training_time = end_time - start_time
        mse.append(history.history['val_loss'][-1])

        y_hat_val_split = model.predict(test_array)
        R2_val.append(r2_score(test_rul, y_hat_val_split))
        RMSE.append(np.sqrt(mean_squared_error(test_rul, y_hat_val_split)))
        score_val.append(compute_s_score(test_rul, y_hat_val_split))
            
        
    #  append results
    d = {'RMSE' :np.mean(RMSE), 'std_RMSE' :np.std(RMSE),
         'S_score' :np.mean(score_val), 'std_S_score' :np.std(score_val),
         'MSE':np.mean(mse), 'std_MSE':np.std(mse),
         'nodes':str(nodes_per_layer), 'dropout':dropout, 
         'activation':activation, 'batch_size':batch_size, 'TW' : sequence_length,
         'time':training_time}

#     results = results.append(pd.DataFrame(d, index=[0]), ignore_index=True)
    results_all003 = pd.concat([results_all003, pd.DataFrame(d, index=[0])], ignore_index=True)
    results_all003.to_csv('results/all/fd003.csv')

(44919, 35, 16) (44919, 1) (259, 35, 16)
Epoch 1/20
1404/1404 [==============================] - 15s 9ms/step - loss: 889.4980 - val_loss: 379.7305
Epoch 2/20
1404/1404 [==============================] - 13s 9ms/step - loss: 288.2106 - val_loss: 228.2756
Epoch 3/20
1404/1404 [==============================] - 13s 9ms/step - loss: 254.8648 - val_loss: 198.0745
Epoch 4/20
1404/1404 [==============================] - 13s 9ms/step - loss: 238.6362 - val_loss: 201.6884
Epoch 5/20
1404/1404 [==============================] - 13s 9ms/step - loss: 227.1073 - val_loss: 178.9996
Epoch 6/20
1404/1404 [==============================] - 13s 9ms/step - loss: 219.1151 - val_loss: 168.1743
Epoch 7/20
1404/1404 [==============================] - 13s 9ms/step - loss: 215.7577 - val_loss: 172.0746
Epoch 8/20
1404/1404 [==============================] - 13s 9ms/step - loss: 207.8927 - val_loss: 176.6565
Epoch 9/20
1404/1404 [==============================] - 13s 9ms/step - loss: 201.3665 - val_loss: 172.8

In [11]:
results_all003

,RMSE,std_RMSE,S_score,std_S_score,MSE,std_MSE,nodes,dropout,activation,batch_size,TW,time
0,12.968202,0.0,773.809820,0.0,185.339279,0.0,[64],0.2,tanh,32,35,131.362647
1,15.317172,0.0,1007.629807,0.0,334.977905,0.0,[64],0.2,tanh,32,35,60.344443
2,13.045141,0.0,724.167435,0.0,176.161057,0.0,[64],0.2,tanh,32,35,106.996501
3,12.709683,0.0,821.695969,0.0,161.536057,0.0,[64],0.2,tanh,32,35,231.366024
4,12.722547,0.0,684.642265,0.0,185.609589,0.0,[64],0.2,tanh,32,35,212.687683


### FD001

In [12]:
# Load data
# Load data and preprocess
train, test, y_test = prepare_data('FD001.txt')
print(train.shape, test.shape, y_test.shape)
sensor_names = ['T20','T24','T30','T50','P20','P15','P30','Nf','Nc','epr','Ps30','phi',
                    'NRf','NRc','BPR','farB','htBleed','Nf_dmd','PCNfR_dmd','W31','W32']
remaining_sensors = ['T24','T30','T50', 'P15', 'P30','Nf','Nc','Ps30','phi',
                    'NRf','NRc','BPR','htBleed','W31','W32']
drop_sensors = [element for element in sensor_names if element not in remaining_sensors]

rul_piecewise = 125
train['RUL'].clip(upper=rul_piecewise, inplace=True)

(20631, 27) (13096, 26) (100, 1)


In [13]:
%%time
results_all001 = pd.DataFrame()
for SEED in range(5):  
    set_seed(SEED)
    mse = []
    R2_val = []
    RMSE = []
    score_val = []
    
    # parameter's sample
    # weights_file = "weights_file_lstm_optimalmodel_all.h5"
    alpha = 0.1
    sequence_length = 30
    epochs = 20
    nodes_per_layer = [128]
    dropout = 0.2
    activation = 'tanh'
    batch_size = 64
    remaining_sensors = remaining_sensors
    # create model
    input_shape = (sequence_length, len(remaining_sensors))
    model = model_lstm_1layer(input_shape, nodes_per_layer[0], dropout,
                             activation)
    
    # Data prepration
    X_train_interim, X_test_interim = prep_data(train, test, drop_sensors, remaining_sensors, alpha)

    # create sequences train, test
    train_array = gen_data_wrapper(X_train_interim, sequence_length,remaining_sensors)
    label_array = gen_label_wrapper(X_train_interim, sequence_length, ['RUL'])

    test_gen = (list(gen_test_data(X_test_interim[X_test_interim['Unit']==unit_nr], sequence_length,remaining_sensors, -99.))
               for unit_nr in X_test_interim['Unit'].unique())
    
    test_array = np.concatenate(list(test_gen)).astype(np.float32)
    test_rul = rul_piecewise_fct(y_test,rul_piecewise)
    print(train_array.shape, label_array.shape, test_array.shape)
            
    # Model fitting
    cb = EarlyStopping(monitor='val_loss', patience=4, restore_best_weights=True)
    with tf.device('/device:GPU:0'):
        start_time = time.time()
        model.compile(loss='mean_squared_error', optimizer=Adam(learning_rate=0.001))
        weights_file = model.get_weights()
        model.set_weights(weights_file)  # reset optimizer and node weights before every training iteration
        history = model.fit(train_array, label_array,
                                validation_data=(test_array, test_rul),
                                epochs=epochs,
                                batch_size=batch_size,
                                callbacks=[cb],
                                verbose=1)
        end_time = time.time()
        training_time = end_time - start_time
        mse.append(history.history['val_loss'][-1])

        y_hat_val_split = model.predict(test_array)
        R2_val.append(r2_score(test_rul, y_hat_val_split))
        RMSE.append(np.sqrt(mean_squared_error(test_rul, y_hat_val_split)))
        score_val.append(compute_s_score(test_rul, y_hat_val_split))
            
        
    #  append results
    d = {'RMSE' :np.mean(RMSE), 'std_RMSE' :np.std(RMSE),
         'S_score' :np.mean(score_val), 'std_S_score' :np.std(score_val),
         'MSE':np.mean(mse), 'std_MSE':np.std(mse),
         'nodes':str(nodes_per_layer), 'dropout':dropout, 
         'activation':activation, 'batch_size':batch_size, 'TW' : sequence_length,
         'time':training_time}

#     results = results.append(pd.DataFrame(d, index=[0]), ignore_index=True)
    results_all001 = pd.concat([results_all001, pd.DataFrame(d, index=[0])], ignore_index=True)
    results_all001.to_csv('results/all/fd001.csv')

(17731, 30, 15) (17731, 1) (100, 30, 15)
Epoch 1/20
278/278 [==============================] - 7s 18ms/step - loss: 1945.0713 - val_loss: 1142.2662
Epoch 2/20
278/278 [==============================] - 4s 16ms/step - loss: 594.9089 - val_loss: 336.2231
Epoch 3/20
278/278 [==============================] - 4s 16ms/step - loss: 349.9518 - val_loss: 213.0034
Epoch 4/20
278/278 [==============================] - 4s 16ms/step - loss: 243.1687 - val_loss: 380.7289
Epoch 5/20
278/278 [==============================] - 4s 16ms/step - loss: 222.0929 - val_loss: 180.0421
Epoch 6/20
278/278 [==============================] - 4s 16ms/step - loss: 203.6436 - val_loss: 266.0940
Epoch 7/20
278/278 [==============================] - 4s 16ms/step - loss: 200.1159 - val_loss: 201.9042
Epoch 8/20
278/278 [==============================] - 4s 16ms/step - loss: 189.3215 - val_loss: 217.6176
Epoch 9/20
278/278 [==============================] - 4s 16ms/step - loss: 187.8194 - val_loss: 170.0289
Epoch 10/20


In [14]:
results_all001

,RMSE,std_RMSE,S_score,std_S_score,MSE,std_MSE,nodes,dropout,activation,batch_size,TW,time
0,12.860256,0.0,286.252775,0.0,184.406876,0.0,[128],0.2,tanh,64,30,69.097678
1,12.859582,0.0,229.533913,0.0,264.980011,0.0,[128],0.2,tanh,64,30,70.237192
2,12.886112,0.0,220.296867,0.0,198.042267,0.0,[128],0.2,tanh,64,30,58.060213
3,13.227489,0.0,262.783810,0.0,307.228760,0.0,[128],0.2,tanh,64,30,67.580328
4,12.654454,0.0,237.325803,0.0,256.517761,0.0,[128],0.2,tanh,64,30,48.572393
